In [0]:
#참고: https://programmers.co.kr/learn/courses/21/lessons/1694

#!pip show BeautifulSoup4

In [145]:
import numpy as np

train=[]
train.append(["<a>gd test 1234</a>"])
train.append(["<p>going to bed, it`s good-time."])
train.append(["I WAS SLEEPED!!!"])
train.append(["this breeding horse breeding horses. +"])
train.append(["coho eggs eyed"])

train_data = np.array(train)
print(train_data)
#for idx, data in enumerate(train_data) : print('train_data',idx, ':', data)

[['<a>gd test 1234</a>']
 ['<p>going to bed, it`s good-time.']
 ['I WAS SLEEPED!!!']
 ['this breeding horse breeding horses. +']
 ['coho eggs eyed']]


In [146]:
#1. html 제거
from bs4 import BeautifulSoup

#example1 = BeautifulSoup(train['review'][0], "html5lib")
#example1.get_text()[:700]

for idx, words in enumerate(train_data) : 
    html_removed = BeautifulSoup(''.join(words)).get_text()
    train_data[idx] = [html_removed]

train_data_h = train_data.copy()
print(train_data_h)

[['gd test 1234']
 ['going to bed, it`s good-time.']
 ['I WAS SLEEPED!!!']
 ['this breeding horse breeding horses. +']
 ['coho eggs eyed']]


In [147]:
#2. 정규표현식을 사용해서 특수문자를 제거, 소문자변환
import re

for idx, words in enumerate(train_data_h) : 
    # 소문자와 대문자가 아닌 것은 공백으로 대체한다.
    #letters_only = re.sub('[^a-zA-Z]', ' ', ''.join(data))
    
    p = re.compile('[a-zA-Z0-9.,-]*')   
    word_list = p.findall(''.join(words))

    new_word_list = []
    for word in word_list:
        if(word.strip()==''): continue
        new_word_list.append(word.strip())
    if(len(new_word_list)<=1): new_word_list = []

    train_data_h[idx] = [' '.join(new_word_list).lower()]

train_data_r = train_data_h.copy()
print(train_data_r)

[['gd test 1234']
 ['going to bed, it s good-time.']
 ['i was sleeped']
 ['this breeding horse breeding horses.']
 ['coho eggs eyed']]


In [148]:
#3. 불용어 제거
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stops = set(stopwords.words("english"))
print('stopwords.words: ', stops)
print()

# stopwords 를 제거한 토큰들
#words = [w for w in train_data if not w in stopwords.words('english')]

for idx, words in enumerate(train_data_r) :
    new_words = []
    #new_words = [w for w in words if not w in stops]
    for word in ''.join(words).split():
        if not word in stops: 
          new_words.append(word)
        else: print('stops word:', word)
    train_data_r[idx] = [' '.join(new_words)]

train_data_s = train_data_r.copy()
print(train_data_s)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
stopwords.words:  {'through', 'you', 'such', 'hasn', 'once', "shan't", 'herself', "needn't", 'of', "don't", 're', 'wasn', 'this', 'it', 'i', 'had', 'after', 'on', 'what', 'can', 'being', 'from', 'too', 'both', "couldn't", 'some', 'isn', "you'd", "should've", 'weren', 'and', 'so', 'an', 'for', 'their', "shouldn't", 'which', 'were', 'all', 'doing', 'before', "mustn't", 'be', 'further', 'few', 'during', 'o', 'her', 'should', 'y', 'couldn', "it's", 'that', 'into', 'ain', 'than', "that'll", 'while', "didn't", 'they', 'needn', 'then', 'hadn', 'm', "you'll", 'his', 'whom', 'them', 'yours', "won't", 'he', 'yourself', 'those', 'by', 'over', 'at', 'will', 'any', 'down', 'between', 'here', 'these', "hadn't", 'more', 'each', 'mustn', 'my', 'does', 'did', 't', 'until', 'ma', 'a', 'nor', "aren't", 'the', 'our', 'other', 'am', 'again', 'if', 'just', 'have', "mightn't", 'been', 'didn'

In [149]:
#4. 원형추출(어간추출)
#horse -> hors 바뀌는 버그있음?
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')

for idx, words in enumerate(train_data_s) :
    new_words = []
    for word in ''.join(words).split():
        new_w = stemmer.stem(word)
        new_words.append(new_w)
    train_data_s[idx] = [' '.join(new_words)]

train_data_st = train_data_s.copy()
print(train_data_st)

[['gd test 1234']
 ['go bed, good-time.']
 ['sleep']
 ['breed hors breed horses.']
 ['coho egg eye']]


In [134]:
#5. Lemmatization 음소표기법
# 언어학에서 음소 표기법 (또는 lemmatization)은 단어의 보조 정리 또는 사전 형식에 의해 식별되는 단일 항목으로 분석될 수 있도록 굴절된 형태의 단어를 그룹화하는 과정이다.
# 예를 들어 동음이의어가 문맥에 따라 다른 의미가 있는데

# 1) 배가 맛있다.
# 2) 배를 타는 것이 재미있다.
# 3) 평소보다 두 배로 많이 먹어서 배가 아프다.

# 위에 있는 3개의 문장에 있는 배는 모두 다른 의미가 있다.

# 레마타이제이션은 이때 앞뒤 문맥을 보고 단어의 의미를 식별하는 것이다.
# 영어에서 meet는 meeting으로 쓰였을 때 회의를 뜻하지만, meet일 때는 만나다는 뜻을 갖는데 그 단어가 명사로 쓰였는지 동사로 쓰였는지에 따라 적합한 의미가 있도록 추출하는 것이다.

nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

print(wordnet_lemmatizer.lemmatize('fly'))
print(wordnet_lemmatizer.lemmatize('flies'))

for idx, words in enumerate(train_data_st) :
    new_words = []
    for word in ''.join(words).split():
        new_w = wordnet_lemmatizer.lemmatize(word)
        new_words.append(new_w)
    train_data_st[idx] = [' '.join(new_words)]

train_data_l = train_data_st.copy()
print(train_data_l)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
fly
fly
[['gd test 1234']
 ['go bed, good-time.']
 ['sleep']
 ['breed hors']
 ['coho egg eye']]


In [0]:
def review_to_words( raw_review ):
    # 1. HTML 제거
    review_text = BeautifulSoup(raw_review, 'html.parser').get_text()
    # 2. 영문자가 아닌 문자는 공백으로 변환
    letters_only = re.sub('[^a-zA-Z]', ' ', review_text)
    # 3. 소문자 변환
    words = letters_only.lower().split()
    # 4. 파이썬에서는 리스트보다 세트로 찾는 게 훨씬 빠르다.
    # stopwords 를 세트로 변환한다.
    stops = set(stopwords.words('english'))
    # 5. Stopwords 불용어 제거
    meaningful_words = [w for w in words if not w in stops]
    # 6. 어간추출
    stemming_words = [stemmer.stem(w) for w in meaningful_words]
    # 7. 공백으로 구분된 문자열로 결합하여 결과를 반환
    return( ' '.join(stemming_words) )